# Walk through all streets in a city

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

In [ ]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## Problem description

## Data

[Seattle streets](https://data.seattle.gov/dataset/Street-Network-Database/afip-2mzr/data) from [data.seattle.gov](https://data.seattle.gov/)

### Read the data

In [ ]:
import shapefile
rshp = shapefile.Reader("Street_Network_Database/WGS84/Street_Network_Database.shp")
shapes = rshp.shapes()
records = rshp.records()

In [ ]:
shapes[0].__dict__

{'bbox': [-122.34721334599999, 47.46946425100003, -122.346519546, 47.469471997000035],
 'parts': [0],
 'points': [[-122.34651954599997, 47.46947199700003],
  [-122.34721334599999, 47.46946425100003]],
 'shapeType': 3}

In [ ]:
{k[0]:v for k,v in zip(rshp.fields[1:], records[0])}

{'ACCESS_COD': 1,
 'AGENCY_COD': 1,
 'ARTERIAL_C': 0,
 'CITYCODE': 0,
 'COMPKEY': 0,
 'COMPTYPE': 0,
 'DIVIDED_CO': 1,
 'F_INTR_ID': 21642,
 'GIS_SEG_LE': '1.71624048000e+002',
 'LEGALLOC_C': 1,
 'L_ADRS_FRO': 977,
 'L_ADRS_TO': 999,
 'L_CITY': 'BURIEN',
 'L_STATE': 'WA',
 'L_ZIP': '98166',
 'ORD_PRE_DI': 'SW',
 'ORD_STNAME': 'SW 149TH ST',
 'ORD_STREET': '149TH',
 'ORD_STRE_1': 'ST',
 'ORD_SUF_DI': b'  ',
 'R_ADRS_FRO': 976,
 'R_ADRS_TO': 998,
 'R_CITY': 'BURIEN',
 'R_STATE': 'WA',
 'R_ZIP': '98166',
 'SEGMENT_TY': 1,
 'SHAPE_Leng': '6.93843239173e-004',
 'SNDSEG_UPD': '20040519',
 'SND_FEACOD': 1,
 'SND_ID': 37898,
 'STNAME_ID': 2569,
 'STRUCTURE_': 1,
 'ST_CODE': 0,
 'T_INTR_ID': 21641,
 'UNITID': '0',
 'UNITID2': '0',
 'VEHICLE_US': 1}

### Short example

In [ ]:
streets5 = list(zip(records[:5], shapes[:5]))
streets5[2][1].points

[[-122.31554790099995, 47.511287922000065],
 [-122.31553241799998, 47.51120351700007],
 [-122.31552978999997, 47.51118938700006],
 [-122.31546052299996, 47.51092530900007],
 [-122.31537415499997, 47.510596031000034],
 [-122.31534125099995, 47.51046084500007],
 [-122.31532328399999, 47.51032437400005]]

In [ ]:
from random import randint

import folium
c = streets5[0][1]
print(c.bbox)
map_osm = folium.Map(location=[c.bbox[1], c.bbox[0]])
for rec, shape in streets5:
    d = {k[0]:v for k,v in zip(rshp.fields[1:], rec)}
    map_osm.add_child(folium.Marker([shape.bbox[1], shape.bbox[0]], popup='Start'))
    map_osm.add_child(folium.PolyLine(locations=shape.points, latlon=False, popup=str(d), weight=10))
from pyensae.notebook_helper import folium_html_map
folium_html_map(map_osm, width="50%")

[-122.34721334599999, 47.46946425100003, -122.346519546, 47.469471997000035]


<pyensae.notebook_helper.folium_helper.folium_html_map.<locals>.CustomFoliumMap at 0x227dd826978>